# **k8o5/studio**
*SAVE COPY IN DRIVE*

In [ ]:
# @title
!pip install -q yt_dlp

import os
import json
import base64
import subprocess
import sys
from IPython.display import display, HTML
from google.colab import output
import yt_dlp

# Ensure yt_dlp is available
try:
    import yt_dlp
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "yt_dlp"])
    import yt_dlp

def get_file_list():
    return sorted([f for f in os.listdir('.') if f.endswith('.mp3')])

def fetch_song_handler(fname):
    try:
        with open(fname, "rb") as f:
            enc = base64.b64encode(f.read()).decode('utf-8')
            data_str = f"data:audio/mpeg;base64,{enc}"
            output.eval_js(f"receiveSongData('{fname}', '{data_str}')")
    except Exception as e:
        print(f"Error fetching {fname}: {e}")
        output.eval_js(f"showStatus('Error loading file', false)")

def dl_handler(query):
    output.eval_js("showStatus('Processing...', true)")
    if query.startswith("http") or query.startswith("www"):
        search_query = query
        output.eval_js("showStatus('Downloading Link...', true)")
    else:
        search_query = f"scsearch1:{query}"
        output.eval_js("showStatus('Searching SoundCloud...', true)")

    opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{'key': 'FFmpegExtractAudio','preferredcodec': 'mp3','preferredquality': '192'}],
        'outtmpl': '%(title)s.%(ext)s',
        'quiet': True,
        'noplaylist': True,
        'extractor_args': {'youtube': {'player_client': ['android', 'web']}}
    }

    try:
        with yt_dlp.YoutubeDL(opts) as ydl:
            ydl.download([search_query])
        f = get_file_list()
        output.eval_js(f'updateLib({json.dumps(f)})')
        output.eval_js("showStatus('Import Complete', false)")
    except Exception as e:
        output.eval_js(f"showStatus('Error: Check Console', false)")
        print(f"Download Error: {e}")

def del_handler(fname):
    if os.path.exists(fname):
        os.remove(fname)
        f = get_file_list()
        output.eval_js(f'updateLib({json.dumps(f)})')

output.register_callback('k8o5.dl', dl_handler)
output.register_callback('k8o5.del', del_handler)
output.register_callback('k8o5.fetch', fetch_song_handler)

files = get_file_list()

HTML_CODE = f"""
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>k8o5/studio</title>
<style>
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700;800&display=swap');

:root {{
    --bg: #000000;
    --surf: #0a0a0a;
    --border: #222;
    --text: #fff;
    --acc: #1db954; /* Spotify Green */
}}

* {{ box-sizing: border-box; margin: 0; padding: 0; outline: none; user-select: none; -webkit-user-drag: none; }}

/* 1:1 WINDOW RATIO LAYOUT */
body {{
    background: #111;
    color: var(--text);
    font-family: 'Inter', sans-serif;
    height: 100vh; width: 100vw;
    overflow: hidden;
    display: flex;
    align-items: center;
    justify-content: center;
}}

#app {{
    width: 100%;
    /* Force 1:1 Aspect Ratio relative to height, constrained by viewport */
    height: 100%;
    max-width: 100vh; /* Ensure it stays square */
    aspect-ratio: 1 / 1;

    background: var(--bg);
    display: grid;
    grid-template-rows: 60px 1fr 90px;
    grid-template-columns: 200px 1fr;
    grid-template-areas: "head head" "side main" "play play";
    border: 1px solid var(--border);
    box-shadow: 0 0 50px rgba(0,0,0,0.5);
}}

/* HEADER */
.header {{
    grid-area: head; border-bottom: 1px solid var(--border);
    display: flex; align-items: center; justify-content: space-between;
    padding: 0 20px; background: rgba(0,0,0,0.8); backdrop-filter: blur(10px); z-index: 50;
}}
.brand {{ font-weight: 800; font-size: 16px; color: #fff; }}
.brand span {{ opacity: 0.5; font-weight: 400; }}

.header-search {{ flex: 1; display: flex; align-items: center; justify-content: center; position: relative; padding: 0 20px; }}
.header-search input {{
    width: 100%; max-width: 300px; background: #1a1a1a; border: 1px solid #333; color: #fff;
    padding: 6px 16px 6px 36px; border-radius: 20px; font-size: 12px; transition: 0.2s;
}}
.header-search input:focus {{ border-color: #555; background: #222; }}
.header-search svg {{ position: absolute; left: calc(50% - 135px); pointer-events: none; }}

.status-bar {{
    background: #1a1a1a; padding: 4px 12px; border-radius: 20px;
    font-size: 11px; font-weight: 600; display: flex; align-items: center; gap: 8px;
    opacity: 0; transition: opacity 0.3s;
}}
.status-bar.show {{ opacity: 1; }}

/* SIDEBAR */
.sidebar {{
    grid-area: side; background: var(--surf); border-right: 1px solid var(--border);
    padding: 20px 0; display: flex; flex-direction: column; gap: 2px;
}}
.nav-btn {{
    padding: 10px 20px; cursor: pointer; color: #888;
    font-size: 13px; font-weight: 600; display: flex; align-items: center; gap: 12px; transition: 0.2s;
}}
.nav-btn:hover {{ color: #fff; }}
.nav-btn.active {{ color: #fff; border-left: 3px solid var(--acc); background: rgba(255,255,255,0.03); }}
.nav-btn svg {{ width: 18px; height: 18px; }}

/* MAIN CONTENT (SCROLL FIX) */
.main {{
    grid-area: main;
    position: relative;
    background: #000;
    overflow: hidden;
    display: flex; /* Flex makes child scroll work better */
    flex-direction: column;
}}
.page {{
    display: none;
    flex: 1; /* Take remaining space */
    overflow-y: auto; /* Scroll here */
    padding: 24px;
    z-index: 2;
}}
.page.active {{ display: block; }}

/* Scrollbar Polish */
.page::-webkit-scrollbar {{ width: 8px; }}
.page::-webkit-scrollbar-track {{ background: #000; }}
.page::-webkit-scrollbar-thumb {{ background: #333; border-radius: 4px; }}
.page::-webkit-scrollbar-thumb:hover {{ background: #555; }}

h1 {{ font-size: 24px; font-weight: 700; margin-bottom: 20px; letter-spacing: -0.5px; }}

.grid {{
    display: grid; grid-template-columns: repeat(auto-fill, minmax(130px, 1fr));
    gap: 20px; padding-bottom: 40px;
}}
.card {{ cursor: pointer; display: flex; flex-direction: column; }}
.art-box {{
    width: 100%; aspect-ratio: 1; border-radius: 8px; background: #1a1a1a;
    position: relative; overflow: hidden; margin-bottom: 10px;
    box-shadow: 0 4px 10px rgba(0,0,0,0.4); transition: transform 0.2s;
}}
.card:hover .art-box {{ transform: translateY(-4px); }}
.art-grad {{ width: 100%; height: 100%; display: flex; align-items: center; justify-content: center; }}
.play-overlay {{
    position: absolute; inset: 0; background: rgba(0,0,0,0.4);
    display: flex; align-items: center; justify-content: center; opacity: 0; transition: 0.2s;
}}
.card:hover .play-overlay {{ opacity: 1; }}
.play-circle {{ width: 36px; height: 36px; background: #fff; border-radius: 50%; display: flex; align-items: center; justify-content: center; color: #000; box-shadow: 0 4px 10px rgba(0,0,0,0.5); transform: scale(0.9); transition: 0.2s; }}
.card:hover .play-circle {{ transform: scale(1); }}
.song-title {{
    font-size: 13px; font-weight: 600; color: #fff; line-height: 1.3;
    display: -webkit-box; -webkit-line-clamp: 2; -webkit-box-orient: vertical; overflow: hidden;
}}
.song-sub {{ font-size: 11px; color: #666; margin-top: 2px; }}

/* FX Styles */
.fx-grid {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 16px; }}
.fx-card {{ background: #111; border: 1px solid #222; padding: 20px; border-radius: 8px; }}
.fx-title {{ font-size: 11px; font-weight: 700; color: #666; text-transform: uppercase; margin-bottom: 16px; letter-spacing: 1px; }}
.slider-group {{ margin-bottom: 20px; }}
.slider-info {{ display: flex; justify-content: space-between; margin-bottom: 8px; font-size: 12px; font-weight: 500; color: #aaa; }}
.range {{ -webkit-appearance: none; width: 100%; height: 4px; background: #222; border-radius: 2px; outline: none; }}
.range::-webkit-slider-thumb {{ -webkit-appearance: none; width: 12px; height: 12px; background: #fff; border-radius: 50%; cursor: pointer; }}

/* Import Styles */
.search-box {{ height: 100%; display: flex; flex-direction: column; align-items: center; justify-content: center; text-align: center; }}
.inp {{ width: 100%; max-width: 400px; background: #111; border: 1px solid #333; color: #fff; padding: 14px; border-radius: 8px; margin-bottom: 20px; }}
.btn-dl {{ background: #fff; color: #000; border: none; padding: 12px 28px; border-radius: 24px; font-weight: 700; font-size: 13px; cursor: pointer; }}

/* PLAYER */
.player {{
    grid-area: play; background: #111; border-top: 1px solid var(--border);
    display: grid; grid-template-columns: 200px 1fr 200px; align-items: center; padding: 0 20px; z-index: 100;
}}
.meta {{ display: flex; align-items: center; gap: 12px; }}
.meta-img {{ width: 48px; height: 48px; background: #222; border-radius: 4px; display: flex; align-items: center; justify-content: center; overflow:hidden; flex-shrink: 0; }}
.meta-info {{ display: flex; flex-direction: column; overflow: hidden; }}
.meta-name {{ font-size: 13px; font-weight: 600; color: #fff; white-space: nowrap; overflow: hidden; text-overflow: ellipsis; max-width: 140px; }}
.meta-stat {{ font-size: 11px; color: #666; }}

.ctrl {{ display: flex; flex-direction: column; align-items: center; gap: 6px; width: 100%; }}
.ctrl-btns {{ display: flex; align-items: center; gap: 20px; }}
.c-btn {{ background: none; border: none; color: #aaa; cursor: pointer; transition: 0.2s; display: flex; align-items: center; justify-content: center; }}
.c-btn:hover {{ color: #fff; }}

/* Loop Button Specifics */
.loop-btn {{ position: relative; width: 32px; height: 32px; }}
.loop-btn.active {{ color: var(--acc); }}
.loop-btn.active:hover {{ color: #1ed760; }}
.loop-dot {{
    position: absolute; top: 12px; left: 14px; font-size: 9px; font-weight: 900; color: #000; display: none;
}}
/* SVG tweak for "1" inside loop */
.loop-one-badge {{
    display: none; position: absolute; top: 50%; left: 50%; transform: translate(-50%, -50%);
    font-size: 8px; font-weight: 800; color: #111; z-index: 2;
}}
.loop-btn.one .loop-one-badge {{ display: block; }}
.loop-btn.one svg {{ fill: var(--acc); }}

.play-btn {{ width: 32px; height: 32px; background: #fff; border-radius: 50%; color: #000; display: flex; align-items: center; justify-content: center; transition: 0.2s; }}
.play-btn:hover {{ transform: scale(1.05); }}

.prog-bar {{ width: 100%; max-width: 400px; height: 3px; background: #333; border-radius: 2px; position: relative; cursor: pointer; }}
.prog-fill {{ width: 0%; height: 100%; background: #fff; border-radius: 2px; }}
.prog-bar:hover .prog-fill {{ background: var(--acc); }}
.prog-handle {{ position: absolute; right: -4px; top: -3px; width: 9px; height: 9px; background: #fff; border-radius: 50%; opacity: 0; }}
.prog-bar:hover .prog-handle {{ opacity: 1; }}

#viz {{ position: absolute; top: 0; left: 0; width: 100%; height: 100%; pointer-events: none; opacity: 0.2; mix-blend-mode: screen; z-index: 1; }}

</style>
</head>
<body>

<div id="app">
    <div class="header">
        <div class="brand">k8o5<span>/studio</span></div>
        <div class="header-search">
            <svg width="14" height="14" fill="#666" viewBox="0 0 24 24"><path d="M15.5 14h-.79l-.28-.27C15.41 12.59 16 11.11 16 9.5 16 5.91 13.09 3 9.5 3S3 5.91 3 9.5 5.91 16 9.5 16c1.61 0 3.09-.59 4.23-1.57l.27.28v.79l5 4.99L20.49 19l-4.99-5zm-6 0C7.01 14 5 11.99 5 9.5S7.01 5 9.5 5 14 7.01 14 9.5 11.99 14 9.5 14z"/></svg>
            <input type="text" id="lib-search" placeholder="Filter..." oninput="filterLib()">
        </div>
        <div class="status-bar" id="status"><div class="spinner"></div><span id="stat-txt">Ready</span></div>
    </div>

    <div class="sidebar">
        <div class="nav-btn active" onclick="nav('lib', this)">
            <svg viewBox="0 0 24 24" fill="currentColor"><path d="M4 6H2v14c0 1.1.9 2 2 2h14v-2H4V6zm16-4H8c-1.1 0-2 .9-2 2v12c0 1.1.9 2 2 2h12c1.1 0 2-.9 2-2V4c0-1.1-.9-2-2-2zm0 14H8V4h12v12z"/></svg> Library
        </div>
        <div class="nav-btn" onclick="nav('fx', this)">
            <svg viewBox="0 0 24 24" fill="currentColor"><path d="M3 17v3a2 2 0 002 2h14a2 2 0 002-2v-3a2 2 0 00-2-2H5a2 2 0 00-2 2zM3 6v3a2 2 0 002 2h14a2 2 0 002-2V6a2 2 0 00-2-2H5a2 2 0 00-2-2H5a2 2 0 00-2 2z"/></svg> FX Rack
        </div>
        <div class="nav-btn" onclick="nav('dl', this)">
            <svg viewBox="0 0 24 24" fill="currentColor"><path d="M19 9h-4V3H9v6H5l7 7 7-7zM5 18v2h14v-2H5z"/></svg> Import
        </div>
    </div>

    <div class="main">
        <canvas id="viz"></canvas>
        <div id="p-lib" class="page active">
            <h1>Library</h1>
            <div id="grid" class="grid"></div>
        </div>
        <div id="p-fx" class="page">
            <h1>Audio Effects</h1>
            <div class="fx-grid">
                <div class="fx-card">
                    <div class="fx-title">Playback</div>
                    <div class="slider-group">
                        <div class="slider-info"><span>Speed</span><span id="v-spd">1.0x</span></div>
                        <input type="range" class="range" id="sl-spd" min="0.5" max="2.0" step="0.01" value="1">
                    </div>
                </div>
                <div class="fx-card">
                    <div class="fx-title">Tone</div>
                    <div class="slider-group">
                        <div class="slider-info"><span>Lows</span><span id="v-low">0 dB</span></div>
                        <input type="range" class="range" id="sl-low" min="-20" max="20" step="1" value="0">
                    </div>
                    <div class="slider-group">
                        <div class="slider-info"><span>Highs</span><span id="v-hi">0 dB</span></div>
                        <input type="range" class="range" id="sl-hi" min="-20" max="20" step="1" value="0">
                    </div>
                </div>
                <div class="fx-card">
                    <div class="fx-title">Space</div>
                    <div class="slider-group">
                        <div class="slider-info"><span>Reverb</span><span id="v-rev">0%</span></div>
                        <input type="range" class="range" id="sl-rev" min="0" max="1" step="0.01" value="0">
                    </div>
                </div>
            </div>
        </div>
        <div id="p-dl" class="page">
            <div class="search-box">
                <h2 style="margin-bottom:10px;">Unified Import</h2>
                <input type="text" id="dl-in" class="inp" placeholder="Search or URL...">
                <button class="btn-dl" onclick="dl()">IMPORT TRACK</button>
            </div>
        </div>
    </div>

    <div class="player">
        <div class="meta">
            <div class="meta-img" id="art"><svg width="24" height="24" fill="#333" viewBox="0 0 24 24"><path d="M12 3v9.28a4.39 4.39 0 00-1.5-.28C8.01 12 6 14.01 6 16.5S8.01 21 10.5 21c2.31 0 4.2-1.75 4.45-4H15V6h4V3h-7z"/></svg></div>
            <div class="meta-info"><div class="meta-name" id="t-name">No Track</div><div class="meta-stat" id="t-stat">Idle</div></div>
        </div>
        <div class="ctrl">
            <div class="ctrl-btns">
                <!-- PREV -->
                <button class="c-btn" onclick="prev()"><svg width="16" height="16" fill="currentColor" viewBox="0 0 24 24"><path d="M6 6h2v12H6zm3.5 6l8.5 6V6z"/></svg></button>

                <!-- PLAY -->
                <button class="play-btn" id="bp" onclick="toggle()"><svg width="14" height="14" fill="currentColor" viewBox="0 0 24 24"><path d="M8 5v14l11-7z"/></svg></button>

                <!-- NEXT -->
                <button class="c-btn" onclick="next(true)"><svg width="16" height="16" fill="currentColor" viewBox="0 0 24 24"><path d="M6 18l8.5-6L6 6v12zM16 6v12h2V6h-2z"/></svg></button>

                <!-- LOOP -->
                <button class="c-btn loop-btn" id="btn-loop" onclick="toggleLoop()">
                    <svg width="18" height="18" fill="currentColor" viewBox="0 0 24 24">
                        <path d="M7 7h10v3l4-4-4-4v3H5v6h2V7zm10 10H7v-3l-4 4 4 4v-3h12v-6h-2v4z"/>
                    </svg>
                    <div class="loop-one-badge">1</div>
                </button>
            </div>
            <div class="prog-bar" id="prog"><div class="prog-fill" id="pfill"><div class="prog-handle"></div></div></div>
        </div>
        <div class="vol-wrap">
            <!-- Vol Icon -->
        </div>
    </div>
</div>

<script>
    let files = {json.dumps(files)};
    let audioCache = {{}};
    let idx=-1, playing=false, ac, src, gain, anal, buf, st=0, ot=0, dur=0;
    let loopMode = 0; // 0=Off, 1=All, 2=One
    let low, high, dist, verb, dry, wet;

    const $ = id => document.getElementById(id);

    function init() {{
        if(ac) return;
        ac = new (window.AudioContext || window.webkitAudioContext)();
        gain = ac.createGain();
        anal = ac.createAnalyser(); anal.fftSize = 256;
        low = ac.createBiquadFilter(); low.type="lowshelf"; low.frequency.value=320;
        high = ac.createBiquadFilter(); high.type="highshelf"; high.frequency.value=3200;
        verb = ac.createConvolver();
        dry = ac.createGain();
        wet = ac.createGain();
        let len = 2 * ac.sampleRate;
        let imp = ac.createBuffer(2, len, ac.sampleRate);
        for(let i=0; i<2; i++) {{
            let ch = imp.getChannelData(i);
            for(let j=0; j<len; j++) ch[j] = (Math.random()*2-1)*Math.pow(1-j/len, 2);
        }}
        verb.buffer = imp;
        gain.connect(anal);
        anal.connect(dry); dry.connect(ac.destination);
        anal.connect(wet); wet.connect(verb); verb.connect(ac.destination);
        gain.gain.value = 0.7;
        applyFx();
        vizLoop();
        if('mediaSession' in navigator) {{
            navigator.mediaSession.setActionHandler('play', () => toggle());
            navigator.mediaSession.setActionHandler('pause', () => toggle());
            navigator.mediaSession.setActionHandler('previoustrack', () => prev());
            navigator.mediaSession.setActionHandler('nexttrack', () => next(true));
        }}
    }}

    // LOOP LOGIC
    function toggleLoop() {{
        loopMode = (loopMode + 1) % 3;
        let btn = $('btn-loop');
        btn.classList.remove('active', 'one');

        if(loopMode === 1) {{ // Loop All
            btn.classList.add('active');
        }} else if(loopMode === 2) {{ // Loop One
            btn.classList.add('active', 'one');
        }}
    }}

    async function load(i) {{
        init();
        if(playing) stop(true);
        idx = i;
        let n = files[i];
        $('t-name').innerText = n.replace('.mp3','');
        $('t-stat').innerText = "Buffering...";
        let h = (n.charCodeAt(0)*10)%360;
        $('art').style.background = `linear-gradient(135deg, hsl(${{h}},50%,20%), hsl(${{h}},50%,10%))`;

        if(audioCache[n]) processAudio(audioCache[n]);
        else google.colab.kernel.invokeFunction('k8o5.fetch', [n]);

        if('mediaSession' in navigator) navigator.mediaSession.metadata = new MediaMetadata({{ title: n.replace('.mp3',''), artist: 'k8o5 Studio' }});
    }}

    window.receiveSongData = (fname, base64Str) => {{
        audioCache[fname] = base64Str;
        if(files[idx] === fname) processAudio(base64Str);
    }};

    async function processAudio(base64Str) {{
        try {{
            let b = Uint8Array.from(atob(base64Str.split(',')[1]), c => c.charCodeAt(0)).buffer;
            buf = await ac.decodeAudioData(b);
            dur = buf.duration;
            play(0);
        }} catch(e) {{ $('t-stat').innerText = "Error"; }}
    }}

    function play(off) {{
        src = ac.createBufferSource(); src.buffer = buf;
        src.connect(low); low.connect(high); high.connect(gain);
        applyFx();
        src.start(0, off);
        st = ac.currentTime - off; ot = 0; playing = true;
        $('bp').innerHTML = `<svg width="14" height="14" fill="currentColor" viewBox="0 0 24 24"><path d="M6 19h4V5H6v14zm8-14v14h4V5h-4z"/></svg>`;
        $('t-stat').innerText = "Playing";

        // AUTO ADVANCE LOGIC
        src.onended = () => {{
            if(playing && (ac.currentTime-st)>=dur-0.1) {{
                // Song finished naturally
                if(loopMode === 2) {{
                    play(0); // Repeat One
                }} else {{
                    next(false); // Normal logic
                }}
            }}
        }};
    }}

    function stop(mem) {{
        if(src) {{ src.stop(); src.disconnect(); }}
        if(playing) ot = ac.currentTime - st;
        playing = false; if(!mem) ot = 0;
        $('bp').innerHTML = `<svg width="14" height="14" fill="currentColor" viewBox="0 0 24 24"><path d="M8 5v14l11-7z"/></svg>`;
        $('t-stat').innerText = "Paused";
    }}

    function toggle() {{
        if(idx<0 && files.length) load(0);
        else if(playing) stop(true);
        else if(buf) play(ot);
    }}

    function next(manual) {{
        if(!files.length) return;

        // If Manual click, always go next (unless single track).
        // If Auto (manual=false), check loop mode.

        let nextIdx = idx + 1;

        if (nextIdx >= files.length) {{
            // End of list
            if (loopMode === 1 || (manual && loopMode !== 2)) {{
                load(0); // Wrap around
            }} else if (manual && loopMode === 2) {{
                load(0); // Manual click on repeat one wraps usually
            }} else {{
                stop(); // Stop at end if loop off
            }}
        }} else {{
            load(nextIdx);
        }}
    }}

    function prev() {{ if(files.length) load((idx-1+files.length)%files.length); }}

    function applyFx() {{
        if(!ac) return;
        if(src) src.playbackRate.value = $('sl-spd').value;
        low.gain.value = $('sl-low').value;
        high.gain.value = $('sl-hi').value;
        let r = $('sl-rev').value;
        dry.gain.value = 1 - r; wet.gain.value = r;
        $('v-spd').innerText = $('sl-spd').value + 'x';
    }}
    ['sl-spd','sl-low','sl-hi','sl-rev'].forEach(id => $(id).addEventListener('input', applyFx));

    $('prog').addEventListener('click', e => {{
        if(!buf) return;
        let r = $('prog').getBoundingClientRect();
        stop(true); play(dur * ((e.clientX - r.left)/r.width));
    }});

    function loop() {{
        if(playing) $('pfill').style.width = ((ac.currentTime-st)/dur)*100 + '%';
        requestAnimationFrame(loop);
    }}
    loop();

    const cv = $('viz'), cx = cv.getContext('2d');
    const res = () => {{ cv.width = $('app').clientWidth; cv.height = $('app').clientHeight; }};
    window.onresize = res; setTimeout(res, 500);
    function vizLoop() {{
        requestAnimationFrame(vizLoop);
        if(!anal) return;
        let d = new Uint8Array(anal.frequencyBinCount);
        anal.getByteFrequencyData(d);
        cx.clearRect(0,0,cv.width,cv.height);
        let bw = (cv.width/d.length)*2.5, x=0;
        cx.fillStyle = 'rgba(255,255,255,0.1)';
        for(let i=0; i<d.length; i++) {{
            let h = (d[i]/255)*200;
            cx.fillRect(x, cv.height-h, bw, h);
            x+=bw+1;
        }}
    }}

    function render() {{
        let h = '';
        if(!files.length) h = `<div style="text-align:center;padding:50px;color:#333;grid-column:1/-1">Library Empty</div>`;
        files.forEach((f, i) => {{
            let c = (f.charCodeAt(0)*20)%360;
            h+=`<div class="card" onclick="load(${{i}})">
                <div class="art-box">
                    <div class="art-grad" style="background:linear-gradient(45deg, hsl(${{c}},40%,20%), hsl(${{c}},40%,10%))">
                        <svg width="32" height="32" fill="currentColor" viewBox="0 0 24 24"><path d="M12 3v9.28a4.39 4.39 0 00-1.5-.28C8.01 12 6 14.01 6 16.5S8.01 21 10.5 21c2.31 0 4.2-1.75 4.45-4H15V6h4V3h-7z"/></svg>
                    </div>
                    <div class="play-overlay"><div class="play-circle"><svg width="18" height="18" viewBox="0 0 24 24" fill="currentColor"><path d="M8 5v14l11-7z"/></svg></div></div>
                    <div class="del-btn" onclick="del('${{f}}', event)"><svg width="12" height="12" viewBox="0 0 24 24" fill="currentColor"><path d="M6 19c0 1.1.9 2 2 2h8c1.1 0 2-.9 2-2V7H6v12zM19 4h-3.5l-1-1h-5l-1 1H5v2h14V4z"/></svg></div>
                </div>
                <div class="song-title" title="${{f}}">${{f.replace('.mp3','')}}</div>
                <div class="song-sub">Local</div>
            </div>`;
        }});
        $('grid').innerHTML = h;
    }}

    window.nav = (p, el) => {{
        document.querySelectorAll('.page').forEach(x=>x.classList.remove('active'));
        $('p-'+p).classList.add('active');
        document.querySelectorAll('.nav-btn').forEach(x=>x.classList.remove('active'));
        el.classList.add('active');
    }};

    window.filterLib = () => {{
        let q = $('lib-search').value.toLowerCase();
        document.querySelectorAll('.card').forEach(c => {{
            let t = c.querySelector('.song-title').innerText.toLowerCase();
            c.style.display = t.includes(q) ? 'flex' : 'none';
        }});
    }};

    window.dl = () => {{
        let v = $('dl-in').value;
        if(v) google.colab.kernel.invokeFunction('k8o5.dl', [v]);
    }};
    window.del = (n, e) => {{
        e.stopPropagation();
        if(confirm('Delete?')) google.colab.kernel.invokeFunction('k8o5.del', [n]);
    }};
    window.updateLib = (f) => {{ files=f; render(); }};
    window.showStatus = (m, l) => {{
        $('status').classList.add('show'); $('stat-txt').innerText=m;
        document.querySelector('.spinner').style.display=l?'block':'none';
        if(!l) setTimeout(()=>$('status').classList.remove('show'),3000);
    }};

    render();
</script>
</body>
</html>
"""
display(HTML(HTML_CODE))